In [1]:
require('rstan')
require('tidyverse')

Loading required package: rstan
Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.17.3, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  1.4.2     ✔ purrr   0.2.4
✔ tidyr   0.8.1     ✔ dplyr   0.7.4
✔ readr   1.1.1     ✔ stringr 1.3.0
✔ tibble  1.4.2     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks rstan::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()


### M1, M2 tables

In [3]:
BIOMARKER.DIR <- "../data/1020/m2/"
biomarker_traits <- list.files(BIOMARKER.DIR, pattern="*.RData")
biomarkers <- sapply(biomarker_traits, function(x) strsplit(strsplit(x, "f_m2_",fixed=TRUE)[[1]][[2]], ".RData")[[1]][[1]])
names(biomarkers) <- NULL
biomarkers
m2_biomarkers <- biomarkers

[1] "Alanine_aminotransferase"        "Albumin"                        
 [3] "Alkaline_phosphatase"            "Apolipoprotein_A"               
 [5] "Apolipoprotein_B.adjust.statins" "arm_fp"                         
 [7] "Aspartate_aminotransferase"      "C_reactive_protein"             
 [9] "Calcium"                         "Cholesterol.adjust.statins"     
[11] "Creatinine_in_urine"             "Creatinine"                     
[13] "Cystatin_C"                      "Direct_bilirubin"               
[15] "eGFR"                            "Gamma_glutamyltransferase"      
[17] "Glucose"                         "Glycated_haemoglobin_HbA1c"     
[19] "HDL_cholesterol"                 "IGF_1"                          
[21] "LDL_direct.adjust.statins"       "leg_fp"                         
[23] "Lipoprotein_A"                   "Non_albumin_protein"            
[25] "Phosphate"                       "Potassium_in_urine"             
[27] "SHBG"                            "Sodium_in_urine"                
[29] "Testosterone"                    "Total_bilirubin"                
[31] "Total_protein"                   "Triglycerides"                  
[33] "trunk_fp"                        "Urate"                          
[35] "Urea"                            "Vitamin_D"                      
[37] "whr"

In [16]:
# M2 output
getM2 <- function(trait){
    load(sprintf("../data/1020/m2/f_m2_%s.RData", trait))

    median_vals <- summary(fit2)$summary[,"50%"]
    p <- median_vals[c("pi[1]", "pi[2]", "pi[3]", "pi[4]")]
    sigmasq <- median_vals[c("sigmasq[1]", "sigmasq[2]", "sigmasq[3]", "sigmasq[4]")]

    min_n_eff <- min(summary(fit2)$summary[,"n_eff"])

    rhat_vals <- summary(fit2)$summary[,"Rhat"]
    rhat_present <- rhat_vals[!is.nan(rhat_vals)]
    high_val <- which(abs(rhat_present - 1) > 0.1)
    rhat_dev <- mean((rhat_present - 1))
    if (length(high_val)>0){ 
        print(sprintf("warning for %s, non-convergent rhat", trait)); 
        print(rhat_present[high_val]);
    }
    df <- data.frame(trait, t(unlist(p)), t(unlist(sigmasq)), rhat_dev, min_n_eff) 
    colnames(df) <- c("trait", "pi[1]", "pi[2]", "pi[3]", "pi[4]", "sigmasq[1]", 
                      "sigmasq[2]", "sigmasq[3]", "sigmasq[4]", "rhat_dev", "min_n_eff")
    

return(df)
}

In [17]:
m2.outs <- lapply(biomarkers, getM2)
m2.df <- do.call(rbind, m2.outs)

In [19]:
m2.df <- m2.df %>% mutate(trait=str_replace_all(trait, ".adjust.statins", ""))
fat_dist <- c("arm_fp", "leg_fp", "whr", "trunk_fp")
m2.df %>% filter(!trait %in% fat_dist) %>% write_csv("../data/res_1024/m2_summary_biomarker.csv")
m2.df %>% filter(trait %in% fat_dist) %>% write_csv("../data/res_1024/m2_summary_anthro.csv")

In [31]:
getM1 <- function(trait){
    load(sprintf("../data/1019/m1/f_%s.RData", trait))

    median_vals <- summary(fit1)$summary[,"50%"]
    p <- median_vals[c("pi[1]", "pi[2]")]
    Sigma <- median_vals[c("Sigma[1,1]", "Sigma[1,2]", "Sigma[2,1]", "Sigma[2,2]")]

    min_n_eff <- min(summary(fit1)$summary[,"n_eff"])

    rhat_vals <- summary(fit1)$summary[,"Rhat"]
    rhat_present <- rhat_vals[!is.nan(rhat_vals)]
    high_val <- which(abs(rhat_present - 1) > 0.1)
    rhat_dev <- mean((rhat_present - 1)**2)
    if (length(high_val)>0){ 
        print(sprintf("warning for %s, non-convergent rhat", trait)); 
        print(rhat_present[high_val]);
    }
    df <- data.frame(trait, t(unlist(p)), t(unlist(Sigma)), rhat_dev, min_n_eff) 
    colnames(df) <- c("trait", "pi[1]", "pi[2]", "Sigma[1,1]", "Sigma[1,2]", "Sigma[2,1]", "Sigma[2,2]", "rhat_dev", "min_n_eff")
    

    return(df)
}

In [51]:
load(sprintf("../data/1019/m1/f_%s.RData", trait))
    median_vals <- summary(fit1)$summary[,"50%"]
#median_vals
matrix(median_vals[c("Sigma[1,1]", "Sigma[1,2]", "Sigma[2,1]", "Sigma[2,2]")], 2,2)

0.0003066901,0.0000558534
0.0000558534,0.0007070983


In [42]:
BIOMARKER.DIR1 <- "../data/tmp_posteriors2/"
biomarker_traits1 <- list.files(BIOMARKER.DIR1, pattern="*.RData")
biomarkers1 <- sapply(biomarker_traits1, function(x) strsplit(strsplit(x, "post_",fixed=TRUE)[[1]][[2]], ".RData")[[1]][[1]])
names(biomarkers1) <- NULL
biomarkers1
    


[1] "Alanine_aminotransferase"        "Albumin"                        
 [3] "Alkaline_phosphatase"            "Apolipoprotein_A"               
 [5] "Apolipoprotein_B.adjust.statins" "arm_fp"                         
 [7] "Aspartate_aminotransferase"      "C_reactive_protein"             
 [9] "Calcium"                         "Cholesterol.adjust.statins"     
[11] "Creatinine_in_urine"             "Creatinine"                     
[13] "Cystatin_C"                      "Direct_bilirubin"               
[15] "eGFR"                            "Gamma_glutamyltransferase"      
[17] "Glucose"                         "Glycated_haemoglobin_HbA1c"     
[19] "HDL_cholesterol"                 "IGF_1"                          
[21] "INI20150"                        "INI21001"                       
[23] "INI23099"                        "INI23105"                       
[25] "INI3063"                         "INI3064"                        
[27] "INI4079"                         "INI4080"                        
[29] "INI48"                           "INI49"                          
[31] "INI50"                           "LDL_direct.adjust.statins"      
[33] "leg_fp"                          "Lipoprotein_A"                  
[35] "Non_albumin_protein"             "Phosphate"                      
[37] "Potassium_in_urine"              "SHBG"                           
[39] "Sodium_in_urine"                 "Testosterone"                   
[41] "Total_bilirubin"                 "Total_protein"                  
[43] "Triglycerides"                   "trunk_fp"                       
[45] "Urate"                           "Urea"                           
[47] "Vitamin_D"                       "whr"

In [34]:
m1.outs <- lapply(biomarkers1, getM1)
m1.df <- do.call(rbind, m1.outs)
m1.df

trait,pi[1],pi[2],"Sigma[1,1]","Sigma[1,2]","Sigma[2,1]","Sigma[2,2]",rhat_dev,min_n_eff
Alanine_aminotransferase,0.9278482,7.215175e-02,1.060224e-04,1.059453e-04,1.059453e-04,1.224150e-04,2.904986e-05,612.9041
Albumin,0.9256475,7.435255e-02,1.092278e-04,1.101330e-04,1.101330e-04,1.152883e-04,2.258763e-05,453.5600
Alkaline_phosphatase,0.9737905,2.620950e-02,8.043119e-04,9.013375e-04,9.013375e-04,1.033662e-03,8.503194e-06,680.5562
Apolipoprotein_A,0.9693103,3.068965e-02,4.745250e-04,4.895478e-04,4.895478e-04,5.271595e-04,4.464052e-05,422.4526
Apolipoprotein_B.adjust.statins,0.9843863,1.561375e-02,2.021600e-03,1.843943e-03,1.843943e-03,1.725707e-03,1.250885e-04,527.9685
Apolipoprotein_B,0.9917642,8.235789e-03,2.492967e-03,2.086329e-03,2.086329e-03,1.793566e-03,2.485399e-05,651.4092
arm_fp,0.6385337,3.614663e-01,3.521451e-05,1.083046e-05,1.083046e-05,1.835193e-05,7.121018e-06,446.2397
Aspartate_aminotransferase,0.9465776,5.342243e-02,2.036557e-04,1.964831e-04,1.964831e-04,1.998303e-04,4.192441e-06,541.0748
C_reactive_protein,0.9819493,1.805070e-02,8.238616e-04,7.621694e-04,7.621694e-04,7.641461e-04,4.188237e-06,560.7892
Calcium,0.9465313,5.346871e-02,1.303813e-04,1.347248e-04,1.347248e-04,1.444306e-04,4.720819e-05,491.7538


In [46]:
m1_anthro <- read_csv("../data/aggreg_1020/m1_anthro_1020.csv")
m1_biomarker <- read_csv("../data/aggreg_1020/m1_biomarker_1020.csv")
#head(m1_anthro)

Parsed with column specification:
cols(
  trait.name = col_character(),
  trait = col_character(),
  n = col_integer(),
  `pi[1]` = col_double(),
  `pi[2]` = col_double(),
  `Sigma[1,1]` = col_double(),
  `Sigma[1,2]` = col_double(),
  `Sigma[2,1]` = col_double(),
  `Sigma[2,2]` = col_double(),
  rg.l = col_double(),
  rg.u = col_double(),
  rg.c = col_double(),
  hf.c = col_double(),
  hf.l = col_double(),
  hf.u = col_double(),
  hm.c = col_double(),
  hm.l = col_double(),
  hm.u = col_double()
)
Parsed with column specification:
cols(
  trait.name = col_character(),
  trait = col_character(),
  n = col_integer(),
  `pi[1]` = col_double(),
  `pi[2]` = col_double(),
  `Sigma[1,1]` = col_double(),
  `Sigma[1,2]` = col_double(),
  `Sigma[2,1]` = col_double(),
  `Sigma[2,2]` = col_double(),
  rg.l = col_double(),
  rg.u = col_double(),
  rg.c = col_double(),
  hf.c = col_double(),
  hf.l = col_double(),
  hf.u = col_double(),
  hm.c = col_double(),
  hm.l = col_double(),
  hm.u = col_dou

In [38]:
m1_a_s <- m1_anthro %>% select(trait.name, trait, n, rg.l, rg.u, rg.c, hf.c, hf.l, hm.c, hm.l, hm.u) 
anthro_df <- left_join(m1_a_s, m1.df) %>% select(trait.name, n, "pi[1]", "pi[2]", "Sigma[1,1]", "Sigma[1,2]", "Sigma[2,1]", "Sigma[2,2]", everything())
anthro_df %>% select(-trait) %>%write_csv("../data/res_1024/m1_summary_anthro.csv")

Joining, by = "trait"
Warning message:
“Column `trait` joining character vector and factor, coercing into character vector”

In [49]:
m1_a_b <- m1_biomarker %>% select(trait.name, trait, n, rg.l, rg.u, rg.c, hf.c, hf.l, hm.c, hm.l, hm.u) 
biomarker_df <- left_join(m1_a_b, m1.df) %>% select(trait.name, n, "pi[1]", "pi[2]", "Sigma[1,1]", "Sigma[1,2]", "Sigma[2,1]", "Sigma[2,2]", everything())
biomarker_df %>% select(-trait) %>%write_csv("../data/res_1024/m1_summary_biomarker.csv")

Joining, by = "trait"
Warning message:
“Column `trait` joining character vector and factor, coercing into character vector”

### M2 posteriors ...

In [62]:
source("../mixture_model_scripts/snp_utils.R")
getPosteriorVec <- function(trait){
load(sprintf("../data/1020/m2/f_m2_%s.RData", trait))
   load(sprintf("../data/1020/dat_%s.RData", trait))

    median_vals <- summary(fit2)$summary[,"50%"]
    p <- median_vals[c("pi[1]", "pi[2]", "pi[3]", "pi[4]")]
    sigmasq <- median_vals[c("sigmasq[1]", "sigmasq[2]", "sigmasq[3]", "sigmasq[4]")]

    if (length(sigmasq)==4) {
        Sigma <- matrix(c(sigmasq[3], 0, 0, sigmasq[4]),2,2)
    } else{
        print("error")
    }
 
    # calculate posteriors
    B_dat <- dat$dat$B
    SE_dat <- dat$dat$SE
    N <- dat$dat$N
    
    posteriors <- lapply(1:N, function(i) computePosterior(B_dat[i,], SE_dat[i,], p, sigmasq, Sigma))

 posterior.df <- data.frame(do.call(rbind,posteriors))
      posterior.df$SNP <- dat$snp
        post.df <- posterior.df %>% filter(p2 >= 0.5 | p3 >= 0.5 | p4 >= 0.5) 

post.df2 <- data.frame(apply(post.df, c(1,2), function(x) unlist(x)))
  
#save(posterior_df, file=sprintf("../data/tmp_posteriors2/m2_post_%s.RData", trait))
colnames(posterior.df) <- c("p1", "p2", "p3", "p4")
p.df <- data.frame(dat$p )
    se.df <- data.frame(dat$dat$SE)
    beta.df <- data.frame(dat$dat$B)
    snp.df <- data.frame(dat$snp)
    chr.df <- data.frame(dat$chr)
    full.df <- do.call(cbind, list(snp.df, chr.df, beta.df, se.df, p.df))
    colnames(full.df) <- c("SNP", "CHR", "B.f", "B.m", "SE.f", "SE.m", "P.f", "P.m")
comb.df <- right_join(full.df,post.df2, by="SNP")
comb.df %>% write_csv(sprintf("../data/res_1024/m2/snp_table_%s.txt", trait))

}

Warning message:
“Column `SNP` joining factors with different levels, coercing to character vector”

In [109]:
comb.df2 <- comb.df %>% mutate(category=case_when(
    as.numeric(as.character(p2)) >= 0.8 ~ 2,
    as.numeric(as.character(p3)) >= 0.8 ~ 3,
    as.numeric(as.character(p4)) >= 0.8 ~ 4,
    TRUE ~ 1
))

In [114]:
snps4 <-read.table("../data/1020/m2/snps4_Testosterone.txt", header=TRUE, stringsAsFactors=FALSE)
snps4.2 <- snps4 %>% mutate(category=case_when(
    as.numeric(as.character(p2)) >= 0.8 ~ 2,
    as.numeric(as.character(p3)) >= 0.8 ~ 3,
    as.numeric(as.character(p4)) >= 0.8 ~ 4,
    TRUE ~ 1
))

In [119]:
setdiff(filter(snps4.2, category==2)$SNP, filter(comb.df2, category==2)$SNP)
comb.df2 %>% filter(SNP=="rs45446698")

[1] "rs45446698"

SNP,CHR,B.f,B.m,SE.f,SE.m,P.f,P.m,p1,p2,p3,p4,category
rs45446698,7,-0.324929,0.0186223,9.649408e-05,8.336658e-05,5.02735e-239,0.0413956,2.0495729052099e-232,0.798286924287871,1.54536500216274e-234,0.201713075712128,1


### M2 FDR

In [8]:
snps <- do.call(rbind, lapply(m2_biomarkers, function(trait) 
    read_csv(sprintf("../data/res_1024/m2/snp_table_%s.txt", trait)) %>% mutate(trait=trait)))

Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = col_character(),
  B.f = col_double(),
  B.m = col_double(),
  SE.f = col_double(),
  SE.m = col_double(),
  P.f = col_double(),
  P.m = col_double(),
  p1 = col_double(),
  p2 = col_double(),
  p3 = col_double(),
  p4 = col_double()
)
Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = col_integer(),
  B.f = col_double(),
  B.m = col_double(),
  SE.f = col_double(),
  SE.m = col_double(),
  P.f = col_double(),
  P.m = col_double(),
  p1 = col_double(),
  p2 = col_double(),
  p3 = col_double(),
  p4 = col_double()
)
Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = col_character(),
  B.f = col_double(),
  B.m = col_double(),
  SE.f = col_double(),
  SE.m = col_double(),
  P.f = col_double(),
  P.m = col_double(),
  p1 = col_double(),
  p2 = col_double(),
  p3 = col_double(),
  p4 = col_double()
)
Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = co

Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = col_integer(),
  B.f = col_double(),
  B.m = col_double(),
  SE.f = col_double(),
  SE.m = col_double(),
  P.f = col_double(),
  P.m = col_double(),
  p1 = col_double(),
  p2 = col_double(),
  p3 = col_double(),
  p4 = col_double()
)
Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = col_character(),
  B.f = col_double(),
  B.m = col_double(),
  SE.f = col_double(),
  SE.m = col_double(),
  P.f = col_double(),
  P.m = col_double(),
  p1 = col_double(),
  p2 = col_double(),
  p3 = col_double(),
  p4 = col_double()
)
Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = col_integer(),
  B.f = col_double(),
  B.m = col_double(),
  SE.f = col_double(),
  SE.m = col_double(),
  P.f = col_character(),
  P.m = col_double(),
  p1 = col_double(),
  p2 = col_double(),
  p3 = col_double(),
  p4 = col_double()
)
Parsed with column specification:
cols(
  SNP = col_character(),
  CHR = c

In [27]:
cutoff <- 0.8
calcFDRM2 <- function(trait.name, cutoff){

    f_t <- snps %>% filter(trait==trait.name & p2 > cutoff) 
    m_t <- snps %>% filter(trait==trait.name & p3 > cutoff) 
    s_t <- snps %>% filter(trait==trait.name & p4 > cutoff)
    
    f_fdr <- ifelse(length(f_t$p1) > 5, sum(f_t$p1)/(length(f_t$p2)), NA)
    m_fdr <- ifelse(length(m_t$p1) > 5, sum(m_t$p1)/(length(m_t$p3)), NA)
    s_fdr <- ifelse(length(s_t$p1) > 5, sum(s_t$p1)/(length(s_t$p4)), NA)

    return(data.frame(list("trait"=trait.name, "cutoff"=cutoff, "f"=f_fdr, "m"=m_fdr, "s"=s_fdr)))
}

fdr_tabs <- do.call(rbind, lapply(m2_biomarkers, function(trait) 
                   do.call(rbind, lapply(seq(0.5, 0.9, 0.1), 
                                         function(cutoff) calcFDRM2(trait, cutoff)))))
#fdr_tabs %>% write_csv("../data/res_1024/m2_fdr.csv")
fat_dist <- c("arm_fp", "leg_fp", "whr", "trunk_fp")
fdr_tabs %>% filter(!trait %in% fat_dist) %>% write_csv("../data/res_1024/m2_fdr_biomarker.csv")
fdr_tabs %>% filter(trait %in% fat_dist) %>% write_csv("../data/res_1024/m2_fdr_counts.csv")                      

### New M2 table

In [21]:
# first - counts 
snps2 <- snps %>% mutate(category=case_when(
    as.numeric(as.character(p2)) >= 0.8 ~ 2,
    as.numeric(as.character(p3)) >= 0.8 ~ 3,
    as.numeric(as.character(p4)) >= 0.8 ~ 4,
    TRUE ~ 1
)) %>% select(trait, everything())



In [28]:
snps3 <- snps2 %>% dplyr::filter(category!=1 ) %>% 
group_by(trait, category) %>% count() %>% 
spread(key=category, value=n) %>% rename("num_f"="2", "num_m"="3", "num_shared"="4") %>%
ungroup(trait, category) %>%
mutate(trait=str_replace_all(trait, ".adjust.statins", ""))

snps3[is.na(snps3)] <- 0
head(snps3)
fat_dist <- c("arm_fp", "leg_fp", "whr", "trunk_fp")
snps3 %>% filter(!trait %in% fat_dist) %>% write_csv("../data/res_1024/m2_counts_biomarker.csv")
snps3 %>% filter(trait %in% fat_dist) %>% write_csv("../data/res_1024/m2_counts_anthro.csv")

trait,num_f,num_m,num_shared
Alanine_aminotransferase,1,1,526
Albumin,0,0,401
Alkaline_phosphatase,4,5,1355
Apolipoprotein_A,1,1,958
Apolipoprotein_B,0,0,1224
arm_fp,528,13,3


In [73]:
snps2 %>% filter(category!=1)  %>% filter(! trait %in% fat_dist) %>% group_by(category) %>% count()

category,n
2,146
3,463
4,25880


In [76]:
snps2 %>% filter(trait=="Testosterone" & CHR=="X") %>% group_by(category) %>% count()



category,n
1,15
2,1
3,55
4,1


In [30]:
# now add the other stuff....
require('data.table')
variant_tab <- fread('/oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.new.tsv.gz', data.table=FALSE)
variant_sm <- variant_tab %>% select(ID, POS, REF, ALT, Gene_symbol, maf,  Consequence, LoF, HGVSp)
variant_short <- variant_sm %>% filter(ID %in% snps2$SNP)
x_tab <- read_tsv("../data/x_variant_info.txt")
snp_gene <- read.table("../data/snp_gene_table.txt", sep=" ", header=TRUE)
xxy_snp <- snp_gene %>% filter(snp %in% x_tab$ID)
head(xxy_snp)
x_w_g <- left_join(x_tab, xxy_snp, by=c("ID"="snp"))
reform_x <- x_w_g %>% rename(Gene_symbol=gene, maf=MAF) %>% mutate(HGVSp="", Consequence="", LoF="") %>% select(colnames(variant_short))
var_w_x <- rbind(variant_short, reform_x)


Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Parsed with column specification:
cols(
  ID = col_character(),
  CHR = col_character(),
  POS = col_integer(),
  REF = col_character(),
  ALT = col_character(),
  MAF = col_double()
)


gene,snp
BTK,Affx-34464730
GPRASP1,Affx-34470541
IL1RAPL2,Affx-34480971
COL4A6,Affx-34492707
COL4A6,Affx-34492709
IRS4,Affx-34494619


Warning message:
“Column `ID`/`snp` joining character vector and factor, coercing into character vector”

In [34]:
comb_x <- right_join(var_w_x, snps2, by=c("ID"="SNP")) 
head(comb_x)


out_tab <- comb_x %>% filter(category!=1) %>%
    mutate(trait=str_replace_all(trait, ".adjust.statins", "")) %>%
      mutate(CHR=factor(CHR, levels=c(1:22, "X", "XY"))) %>% # do this to reorder properly
      arrange(trait, CHR, POS) %>%  
      select(trait, ID, CHR, POS, everything()) %>% 
      select(-Consequence, -LoF, -HGVSp, Consequence, LoF, HGVSp) %>%
     rename( "MAF"="maf", "GENE"="Gene_symbol") 



ID,POS,REF,ALT,Gene_symbol,maf,Consequence,LoF,HGVSp,trait,⋯,B.m,SE.f,SE.m,P.f,P.m,p1,p2,p3,p4,category
rs5932715,129147373,G,A,BCORL1,0.89225800,,,,Alanine_aminotransferase,⋯,0.0268672,2.918009e-05,1.682534e-05,0.00103974,5.77272e-11,1.438840e-07,6.416010e-13,4.579966e-06,0.9999953,4
rs73225599,129279023,C,T,AIFM1,0.94836100,,,,Alanine_aminotransferase,⋯,0.0197431,5.699586e-05,3.303031e-05,8.71734e-05,5.92160e-04,4.052661e-02,2.570833e-06,3.188113e-07,0.9594705,4
rs140801471,153542572,G,A,TKTL1,0.95704600,,,,Alanine_aminotransferase,⋯,-0.0294690,7.119191e-05,4.074741e-05,0.446275,3.90524e-06,3.169683e-01,1.362470e-08,3.218811e-04,0.6827098,1
rs28814617,154093855,C,T,F8,0.90703500,,,,Alanine_aminotransferase,⋯,-0.0215671,3.799835e-05,2.163459e-05,0.29332,3.54184e-06,3.666723e-01,1.496560e-08,2.980277e-04,0.6330297,1
rs263526,2173504,T,C,SKI,0.38452375,intron_variant,,,Alanine_aminotransferase,⋯,-0.0140366,1.246062e-05,1.444448e-05,0.00222582,2.21468e-04,4.979614e-01,7.197779e-07,6.498346e-06,0.5020313,1
rs77797313,2186115,G,A,SKI,0.03826131,intron_variant,,,Alanine_aminotransferase,⋯,0.0395107,7.992843e-05,9.296026e-05,0.00389956,4.17058e-05,6.336488e-02,1.390425e-07,1.014367e-05,0.9366248,4


In [37]:
OUT.DIR <- "../data/res_1024/"
out_tab %>% filter(trait %in%  fat_dist & category==2)  %>% select(-category) %>% arrange(trait, CHR) %>% write_csv(sprintf("%s/m2_anthro_f_spec.csv", OUT.DIR))
out_tab %>% filter(trait %in%  fat_dist & category==3)   %>% select(-category) %>%  arrange(trait, CHR) %>% write_csv(sprintf("%s/m2_anthro_m_spec.csv", OUT.DIR))

out_tab %>% filter(trait %in% fat_dist & category==4)   %>% select(-category) %>%  arrange(trait, CHR) %>% write_csv(sprintf("%s/m2_anthro_shared.csv", OUT.DIR))

out_tab %>% filter(category==2 & (! trait %in% fat_dist))  %>% select(-category) %>%  arrange(trait, CHR) %>% write_csv(sprintf("%s/m2_biomarker_f_spec.csv", OUT.DIR))
out_tab %>% filter(category==3& (! trait %in% fat_dist))  %>% select(-category) %>%  arrange(trait, CHR) %>% write_csv(sprintf("%s/m2_biomarker_m_spec.csv", OUT.DIR))
out_tab %>% filter(category==4& (! trait %in% fat_dist))  %>% select(-category) %>%  arrange(trait, CHR) %>% write_csv(sprintf("%s/m2_biomarker_shared.csv", OUT.DIR))

### M1 posteriors... 

In [52]:
require('rstan')
require('tidyverse')
require('MASS')
require('mnormt')

getPosterior2 <- function(B, SE, p, Sigma){
        # get the posterior probability for a SNP

    zeros <- rep(0, length(SE)) #c(0,0)
    SE_mat <- diag(SE) #matrix(c(SE[1], 0, 0, SE[2]), 2, 2)
    p_1 = p[1]*dmnorm(B, zeros, SE_mat)
    p_2 = p[2]*dmnorm(B, zeros, SE_mat + Sigma)
    prob_1 = log(p_1) - log(p_1 + p_2)
    prob_2 = log(p_2) - log(p_1 + p_2)
    return(exp(prob_2))
}


getPosteriorVec <- function(trait){
    load(sprintf("../data/1019/m1/f_%s.RData", trait))
    load(sprintf("../data/1019/dat_%s.RData", trait))
    
    median_vals <- summary(fit1)$summary[,"50%"]
    p <- as.vector(median_vals[c("pi[1]", "pi[2]")])


    Sigma <- matrix(median_vals[c("Sigma[1,1]", "Sigma[1,2]", "Sigma[2,1]", "Sigma[2,2]")], 2,2)
    B_dat <- dat$dat$B
    SE_dat <- dat$dat$SE
    N <- dat$dat$N
    print(p)
    print(Sigma)

    posteriors <- sapply(1:N, function(i) getPosterior2(B_dat[i,], SE_dat[i,], p, Sigma))
    save(posteriors, file=sprintf("../data/tmp_posteriors2/post_%s.RData", trait))
    return(posteriors)
}

Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: mnormt


### M1 FDR

In [43]:
calcFDRM1 <- function(post_vec, cutoff){
    # for M1
    non_null <- post_vec[post_vec > cutoff]
    if (length(non_null) < 5){
        return(NA)
    }
    return(sum(1-non_null)/length(non_null))
}


post_lists <- lapply(biomarkers1, function(trait) {
    load(sprintf("../data/tmp_posteriors2/post_%s.RData", trait));
    return(posteriors)
    })



In [44]:
fdrs <- do.call(cbind,
        lapply(seq(0.5, 0.9, 0.1), function(cutoff) 
            data.frame(sapply(post_lists, function(x) calcFDRM1(x, cutoff)))))

colnames(fdrs) <- sapply(seq(0.5, 0.9, 0.1), function(x) sprintf("post%s", x))
fdrs$trait <- biomarkers1

In [48]:
m1_anthro %>% select(trait.name, trait)

trait.name,trait
arm fp,arm_fp
FEV-1,INI20150
BMI,INI21001
body fat percent,INI23099
basal metabolic rate,INI23105
FVC,INI3063
PEF,INI3064
BP-diastolic,INI4079
BP-systolic,INI4080
INI48,INI48


In [47]:
fdrs %>% dplyr::select(trait, everything()) %>% filter(trait %in% m1_anthro$trait)  %>%
write_csv("../data/res_1024/m1_anthro_fdr.csv")

fdrs %>% dplyr::select(trait, everything()) %>% filter(!trait %in% m1_anthro$trait)  %>%
write_csv("../data/res_1024/m1_biomarker_fdr.csv")

### M1 meno Pi estimates

In [52]:
getM1_3 <- function(trait){
    load(sprintf("../data/1019/d3/m1/f_%s.RData", trait))

    median_vals <- summary(fit1)$summary[,"50%"]
    p <- median_vals[c("pi[1]", "pi[2]")]

    min_n_eff <- min(summary(fit1)$summary[,"n_eff"])

    rhat_vals <- summary(fit1)$summary[,"Rhat"]
    rhat_present <- rhat_vals[!is.nan(rhat_vals)]
    high_val <- which(abs(rhat_present - 1) > 0.1)
    rhat_dev <- mean((rhat_present - 1)**2)
    if (length(high_val)>0){ 
        print(sprintf("warning for %s, non-convergent rhat", trait)); 
        print(rhat_present[high_val]);
    }
    df <- data.frame(trait, t(unlist(p)), rhat_dev, min_n_eff) 
    colnames(df) <- c("trait", "pi[1]", "pi[2]", "rhat_dev", "min_n_eff")
    

    return(df)
}

In [50]:
BIOMARKER.DIR <- "../data/1019/d3/m1/"
biomarker_traits <- list.files(BIOMARKER.DIR, pattern="*.RData")
biomarkers <- sapply(biomarker_traits, function(x) strsplit(strsplit(x, "f_",fixed=TRUE)[[1]][[2]], ".RData")[[1]][[1]])
names(biomarkers) <- NULL
biomarkers
meno_biomarkers <- biomarkers

[1] "Alanine_aminotransferase"        "Albumin"                        
 [3] "Alkaline_phosphatase"            "Apolipoprotein_A"               
 [5] "Apolipoprotein_B.adjust.statins" "Apolipoprotein_B"               
 [7] "Aspartate_aminotransferase"      "C_reactive_protein"             
 [9] "Calcium"                         "Cholesterol.adjust.statins"     
[11] "Creatinine_in_urine"             "Creatinine"                     
[13] "Cystatin_C"                      "Direct_bilirubin"               
[15] "eGFR"                            "Gamma_glutamyltransferase"      
[17] "Glucose"                         "Glycated_haemoglobin_HbA1c"     
[19] "HDL_cholesterol"                 "IGF_1"                          
[21] "LDL_direct.adjust.statins"       "Lipoprotein_A"                  
[23] "Non_albumin_protein"             "Phosphate"                      
[25] "Potassium_in_urine"              "SHBG"                           
[27] "Sodium_in_urine"                 "Testosterone"                   
[29] "Total_bilirubin"                 "Total_protein"                  
[31] "Triglycerides"                   "Urate"                          
[33] "Urea"                            "Vitamin_D"

In [53]:
meno.df <- do.call(rbind, lapply(meno_biomarkers, getM1_3))

[1] "warning for Potassium_in_urine, non-convergent rhat"
        pi[1]         pi[2]  L_Omega[2,1]  L_Omega[2,2]  L_Omega[3,1] 
     7.597569      7.597569      1.987593      3.203208      1.776148 
 L_Omega[3,2]  L_Omega[3,3] Omegacor[1,2] Omegacor[1,3] Omegacor[2,1] 
     1.193722      1.965887      1.987593      1.776148      1.987593 
Omegacor[2,3] Omegacor[3,1] Omegacor[3,2]          lp__ 
     1.921046      1.776148      1.921046    190.301629 


In [57]:
meno.df 

trait,pi[1],pi[2],rhat_dev,min_n_eff
Alanine_aminotransferase,0.9318944,0.068105631,3.944909e-06,630.2483
Albumin,0.9315010,0.068499004,1.100820e-05,389.4354
Alkaline_phosphatase,0.9776712,0.022328780,1.274969e-05,615.6655
Apolipoprotein_A,0.9721931,0.027806852,3.638139e-06,573.0170
Apolipoprotein_B.adjust.statins,0.9867992,0.013200829,1.712575e-05,407.0225
Apolipoprotein_B,0.9927658,0.007234232,4.948122e-06,549.6481


In [58]:
meno_prev <- read_csv("../data/aggreg_1020/m1_menopause_results.csv")

Parsed with column specification:
cols(
  trait = col_character(),
  `pi[2]` = col_double(),
  rg.pre_post = col_double(),
  rg.pre_post.l = col_double(),
  rg.pre_post.u = col_double(),
  rg.pre_male = col_double(),
  rg.pre_male.l = col_double(),
  rg.pre_male.u = col_double(),
  rg.post_male = col_double(),
  rg.post_male.l = col_double(),
  rg.post_male.u = col_double()
)


In [64]:
meno_prev %>% rename("p2"="pi[2]") %>% select(-p2)

trait,rg.pre_post,rg.pre_post.l,rg.pre_post.u,rg.pre_male,rg.pre_male.l,rg.pre_male.u,rg.post_male,rg.post_male.l,rg.post_male.u
Alanine_aminotransferase,0.9800086,0.9597647,0.9935363,0.8717186,0.83696414,0.9046926,0.92788546,0.90886945,0.9455862
Albumin,0.9714352,0.9443273,0.9929576,0.9464726,0.90960686,0.9784959,0.98055746,0.96413327,0.9939114
Alkaline_phosphatase,0.9869555,0.9798687,0.9930635,0.9833978,0.97729655,0.9889923,0.98815539,0.98467750,0.9915207
Apolipoprotein_A,0.9963032,0.9912052,0.9992437,0.9730816,0.96268292,0.9825325,0.97584741,0.96895333,0.9819257
Apolipoprotein_B,0.9982702,0.9961970,0.9996392,0.9838716,0.98004757,0.9873768,0.98622941,0.98344709,0.9888099
Aspartate_aminotransferase,0.9950762,0.9880753,0.9991947,0.9634625,0.94501611,0.9793297,0.97156945,0.96100220,0.9816227
C_reactive_protein,0.9952803,0.9903621,0.9986461,0.9426576,0.93172460,0.9527273,0.95694590,0.94915352,0.9641301
Calcium,0.9831297,0.9602868,0.9966664,0.9514626,0.92085168,0.9777368,0.96414768,0.94367849,0.9829340
Cholesterol,0.9972298,0.9940877,0.9993663,0.9655673,0.95826926,0.9718722,0.97432531,0.96963254,0.9787952
Creatinine,0.9873551,0.9717288,0.9969339,0.9034593,0.87760992,0.9274666,0.93345575,0.91826006,0.9486399


In [67]:
meno_full <- inner_join(meno.df %>% select(trait, "pi[2]"), meno_prev %>% rename("p2"="pi[2]") %>% select(-p2))

Joining, by = "trait"
Warning message:
“Column `trait` joining factor and character vector, coercing into character vector”

In [69]:
meno_full %>% write_csv("../data/res_1024/m1_meno.csv")